In [3]:
import psutil
import platform

print("### Hardware Requirements ###")
print(f"CPU: {platform.processor()}")
print(f"Total RAM: {psutil.virtual_memory().total / 1e9:.2f} GB")

### Hardware Requirements ###
CPU: x86_64
Total RAM: 810.20 GB


In [4]:
import tensorflow as tf
import pandas as pd
import numpy as np
import platform

print("### Software Requirements ###")
print(f"Python Version: {platform.python_version()}")
print(f"TensorFlow Version: {tf.__version__}")
print(f"Pandas Version: {pd.__version__}")
print(f"Numpy Version: {np.__version__}")

2025-03-11 13:16:39.960530: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-11 13:16:52.678890: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


### Software Requirements ###
Python Version: 3.9.13
TensorFlow Version: 2.12.0
Pandas Version: 1.5.0
Numpy Version: 1.24.0


In [5]:
import time
start_time = time.time()

# BioMapAI Tutorial

This tutorial will guide you through:
1. **Environment Setup**  
2. **Data Preparation**  
3. **Model Initialization and Training**  
4. **Prediction and Evaluation**  
5. **Score Conversion and Weight Adjustment**  

By the end, you should be able to train your own model on custom data.


## 1. Environment Setup

1. **Download or Clone** the repository/project that contains:
   - **BioMapAI.py** (the script with model classes and methods)
   - **example_data/** (containing your `train_data.csv` and `test_data.csv`)
   - A script or notebook for your tutorial (this file)

2. **Install Required Libraries** (if you haven’t already):
```bash
pip install numpy pandas tensorflow
```

3. **Import modules** in Python:
```python
import pandas as pd
import numpy as np
import random
import tensorflow as tf
import importlib.util
import os

# Clear any previous TensorFlow session
tf.keras.backend.clear_session()
```

4. **Function to import `BioMapAI.py`**:
```python
def import_module_with_full_path(file_path):
    base_filename = os.path.basename(file_path)
    module_name = os.path.splitext(base_filename)[0]
    module_spec = importlib.util.spec_from_file_location(module_name, file_path)
    imported_module = importlib.util.module_from_spec(module_spec)
    module_spec.loader.exec_module(imported_module)
    return imported_module
```


## 2. Data Preparation

Let’s assume you have:
- A **training dataset** named `train_data.csv`
- A **test dataset** named `test_data.csv`

Each row might look like:

| y0 (label) | Y1 | Y2 | ... | Y12 | X1 | X2 | ... | Xn |
|------------|----|----|-----|-----|----|----|-----|----|
| 0 or 1     | …  | …  | …   | …   | …  | …  | …   | …  |

Where:
- **y0** is your single-class label (e.g., 0 or 1)
- **Y** columns (Y1–Y12) are intermediate targets
- **X** columns (X1–Xn) are input features

### Load Your Data
```python
# 1) Import the BioMapAI module
BioMapAI = import_module_with_full_path("BioMapAI.py")

# 2) Load training data
data = pd.read_csv("example_data/train_data.csv", index_col=0)
y0 = data.iloc[:, 0]       # Single-class label
Y = data.iloc[:, 1:13]     # Intermediate target columns
X = data.iloc[:, 13:]      # Input features

# 3) Load test data
test_data = pd.read_csv("example_data/test_data.csv", index_col=0)
y0_test = test_data.iloc[:, 0]
Y_test = test_data.iloc[:, 1:13]
X_test = test_data.iloc[:, 13:]
```


## 3. Model Initialization and Training

### 3.1 **OmicScoreModel**
The **OmicScoreModel** is trained to predict the intermediate targets `Y`.

```python
model, model_history = BioMapAI.OmicScoreModel(
    epochs=500, 
    optimizer=tf.keras.optimizers.Adam(0.0005), 
    batch_size=64, 
    kernel_regularizer=tf.keras.regularizers.L2(0.008), 
    model_name="example"
).train(X, Y)
```
- **epochs**: Number of training iterations.
- **optimizer**: Choice of optimizer and learning rate.
- **batch_size**: Batch size for training.
- **kernel_regularizer**: Regularization to avoid overfitting.
- **model_name**: Useful if you want to save or track multiple models.

### 3.2 Monitoring the Training History
```python
print(model_history.history.keys())
# You can plot loss curves, accuracy, etc.
```


## 4. Prediction and Evaluation

### 4.1 Predict Intermediate Targets `Y`
```python
Y_pred = BioMapAI.OmicScoreModel().predict(model, X_test)
```
This outputs the intermediate predictions for each of the 12 `Y` columns.

### 4.2 Evaluate Intermediate Target Predictions
Compute the Mean Squared Error (MSE) on `Y_test`:
```python
mse = BioMapAI.OmicScoreModel().score(model, X_test, Y_test)
print("Intermediate Target MSE:", mse)
```


## 5. Score Conversion and Weight Adjustment

### 5.1 Build a Layer for Converting `Y` to `y0`
```python
Y_to_y_model = BioMapAI.ScoreLayer().build_model(Y, y0)
```
This creates a secondary model that maps the 12-dimensional `Y` to a single label or score.

### 5.2 Predict Final Labels `y0`
```python
y_pred = BioMapAI.ScoreYModel(model, Y_to_y_model).predict(X_test)
```
### 5.3 Evaluate Final Model
```python
loss, accuracy = BioMapAI.ScoreYModel(model, Y_to_y_model).evaluate(X_test, y0_test)
print("Final Model Loss:", loss)
print("Final Model Accuracy:", accuracy)
```

### 5.4 Adjusting Weights (Optional)
If you want to fine-tune how the intermediate predictions map to the final label:
```python
Y_to_y_model_adjust = BioMapAI.WeightsAdjust(model, X, Y, Y_to_y_model).adjust_score_weight()
```
Then re-predict and re-evaluate:
```python
y_pred = BioMapAI.ScoreYModel(model, Y_to_y_model).predict(X_test)
loss, accuracy = BioMapAI.ScoreYModel(model, Y_to_y_model).evaluate(X_test, y0_test)
print("Adjusted Final Model Loss:", loss)
print("Adjusted Final Model Accuracy:", accuracy)
```


## Putting It All Together

Below is a compact version of the entire flow. You can copy this code into its own file (e.g., `train_and_eval.py`) or run it as a Jupyter cell.


In [6]:
#!/usr/bin/env python
# coding: utf-8

import pandas as pd
import numpy as np
import random
import tensorflow as tf
tf.keras.backend.clear_session()

import importlib.util
import os

def import_module_with_full_path(file_path):
    base_filename = os.path.basename(file_path)
    module_name = os.path.splitext(base_filename)[0]
    module_spec = importlib.util.spec_from_file_location(module_name, file_path)
    imported_module = importlib.util.module_from_spec(module_spec)
    module_spec.loader.exec_module(imported_module)
    return imported_module

# 1) Import BioMapAI module
BioMapAI = import_module_with_full_path("BioMapAI.py")

# 2) Load data
data = pd.read_csv("example_data/train_data.csv", index_col=0)
y0 = data.iloc[:, 0]
Y = data.iloc[:, 1:13]
X = data.iloc[:, 13:]

test_data = pd.read_csv("example_data/test_data.csv", index_col=0)
y0_test = test_data.iloc[:, 0]
Y_test = test_data.iloc[:, 1:13]
X_test = test_data.iloc[:, 13:]

# 3) Train OmicScoreModel
model, model_history = BioMapAI.OmicScoreModel(
    epochs=500,
    optimizer=tf.keras.optimizers.Adam(0.0005),
    batch_size=64,
    kernel_regularizer=tf.keras.regularizers.L2(0.008),
    model_name="example"
).train(X, Y)

# 4) Predict and evaluate intermediate targets
Y_pred = BioMapAI.OmicScoreModel().predict(model, X_test)
mse = BioMapAI.OmicScoreModel().score(model, X_test, Y_test)
print("Intermediate Target MSE:", mse)

# 5) Build layer to convert Y --> y0
Y_to_y_model = BioMapAI.ScoreLayer().build_model(Y, y0)

# 6) Predict final output
y_pred = BioMapAI.ScoreYModel(model, Y_to_y_model).predict(X_test)

# 7) Evaluate final output
loss, accuracy = BioMapAI.ScoreYModel(model, Y_to_y_model).evaluate(X_test, y0_test)
print("Initial Final Model Loss:", loss)
print("Initial Final Model Accuracy:", accuracy)

# 8) (Optional) Adjust weights
Y_to_y_model_adjust = BioMapAI.WeightsAdjust(model, X, Y, Y_to_y_model).adjust_score_weight()

# 9) Predict & Evaluate again
y_pred = BioMapAI.ScoreYModel(model, Y_to_y_model).predict(X_test)
loss, accuracy = BioMapAI.ScoreYModel(model, Y_to_y_model).evaluate(X_test, y0_test)
print("Adjusted Final Model Loss:", loss)
print("Adjusted Final Model Accuracy:", accuracy)

Epoch 1/500
3/3 [==============================] - 9s 16ms/step - loss: 8.1794 - out0_loss: 0.1110 - out1_loss: 2.0877 - out2_loss: 1.5704 - out3_loss: 0.2588 - out4_loss: 0.0871 - out5_loss: 0.2715 - out6_loss: 0.4267 - out7_loss: 0.1504 - out8_loss: 0.2494 - out9_loss: 0.2274 - out10_loss: 0.1452 - out11_loss: 0.3920 - out0_mse: 0.2219 - out1_accuracy: 0.2000 - out2_accuracy: 0.2629 - out3_mse: 0.5176 - out4_mse: 0.1871 - out5_mse: 0.8562 - out6_mse: 1.0699 - out7_mse: 0.3008 - out8_mse: 0.6876 - out9_mse: 0.5577 - out10_mse: 0.3040 - out11_mse: 1.0433
Epoch 2/500
3/3 [==============================] - 0s 10ms/step - loss: 8.0260 - out0_loss: 0.1110 - out1_loss: 2.1277 - out2_loss: 1.5105 - out3_loss: 0.2784 - out4_loss: 0.1011 - out5_loss: 0.2507 - out6_loss: 0.4155 - out7_loss: 0.1511 - out8_loss: 0.2297 - out9_loss: 0.1909 - out10_loss: 0.1131 - out11_loss: 0.3538 - out0_mse: 0.2219 - out1_accuracy: 0.2000 - out2_accuracy: 0.2971 - out3_mse: 0.5612 - out4_mse: 0.2125 - out5_mse: 0

3/3 [==============================] - 0s 10ms/step - loss: 7.2926 - out0_loss: 0.1110 - out1_loss: 1.8307 - out2_loss: 1.4306 - out3_loss: 0.2520 - out4_loss: 0.0916 - out5_loss: 0.2267 - out6_loss: 0.3221 - out7_loss: 0.1429 - out8_loss: 0.1839 - out9_loss: 0.1860 - out10_loss: 0.1162 - out11_loss: 0.3122 - out0_mse: 0.2219 - out1_accuracy: 0.2343 - out2_accuracy: 0.3486 - out3_mse: 0.5098 - out4_mse: 0.2063 - out5_mse: 0.5046 - out6_mse: 0.7610 - out7_mse: 0.2859 - out8_mse: 0.4298 - out9_mse: 0.3781 - out10_mse: 0.2453 - out11_mse: 0.7456
Epoch 16/500
3/3 [==============================] - 0s 10ms/step - loss: 7.0696 - out0_loss: 0.1110 - out1_loss: 1.7947 - out2_loss: 1.4665 - out3_loss: 0.2543 - out4_loss: 0.0694 - out5_loss: 0.1744 - out6_loss: 0.2455 - out7_loss: 0.1562 - out8_loss: 0.1641 - out9_loss: 0.1790 - out10_loss: 0.1319 - out11_loss: 0.2435 - out0_mse: 0.2219 - out1_accuracy: 0.2457 - out2_accuracy: 0.3600 - out3_mse: 0.5216 - out4_mse: 0.1428 - out5_mse: 0.4114 - out

3/3 [==============================] - 0s 10ms/step - loss: 6.4618 - out0_loss: 0.1110 - out1_loss: 1.5981 - out2_loss: 1.3830 - out3_loss: 0.2574 - out4_loss: 0.0617 - out5_loss: 0.1707 - out6_loss: 0.1819 - out7_loss: 0.1442 - out8_loss: 0.1204 - out9_loss: 0.1558 - out10_loss: 0.1110 - out11_loss: 0.1822 - out0_mse: 0.2219 - out1_accuracy: 0.2914 - out2_accuracy: 0.3771 - out3_mse: 0.5162 - out4_mse: 0.1234 - out5_mse: 0.3818 - out6_mse: 0.4017 - out7_mse: 0.2885 - out8_mse: 0.2498 - out9_mse: 0.3339 - out10_mse: 0.2221 - out11_mse: 0.4159
Epoch 30/500
3/3 [==============================] - 0s 10ms/step - loss: 6.3285 - out0_loss: 0.1110 - out1_loss: 1.5814 - out2_loss: 1.3003 - out3_loss: 0.2221 - out4_loss: 0.0666 - out5_loss: 0.1844 - out6_loss: 0.1770 - out7_loss: 0.1410 - out8_loss: 0.1302 - out9_loss: 0.1681 - out10_loss: 0.1049 - out11_loss: 0.1640 - out0_mse: 0.2219 - out1_accuracy: 0.3371 - out2_accuracy: 0.4286 - out3_mse: 0.4443 - out4_mse: 0.1355 - out5_mse: 0.4153 - out

3/3 [==============================] - 0s 9ms/step - loss: 5.8115 - out0_loss: 0.1110 - out1_loss: 1.4400 - out2_loss: 1.2804 - out3_loss: 0.1946 - out4_loss: 0.0663 - out5_loss: 0.1220 - out6_loss: 0.1211 - out7_loss: 0.1354 - out8_loss: 0.1011 - out9_loss: 0.1345 - out10_loss: 0.1081 - out11_loss: 0.1067 - out0_mse: 0.2219 - out1_accuracy: 0.3371 - out2_accuracy: 0.4629 - out3_mse: 0.3892 - out4_mse: 0.1341 - out5_mse: 0.2488 - out6_mse: 0.2523 - out7_mse: 0.2708 - out8_mse: 0.2028 - out9_mse: 0.2844 - out10_mse: 0.2162 - out11_mse: 0.2201
Epoch 44/500
3/3 [==============================] - 0s 9ms/step - loss: 5.7736 - out0_loss: 0.1110 - out1_loss: 1.3923 - out2_loss: 1.2254 - out3_loss: 0.2184 - out4_loss: 0.0616 - out5_loss: 0.1303 - out6_loss: 0.1367 - out7_loss: 0.1379 - out8_loss: 0.0979 - out9_loss: 0.1271 - out10_loss: 0.1138 - out11_loss: 0.1375 - out0_mse: 0.2219 - out1_accuracy: 0.4457 - out2_accuracy: 0.4800 - out3_mse: 0.4370 - out4_mse: 0.1299 - out5_mse: 0.2653 - out6_

3/3 [==============================] - 0s 11ms/step - loss: 5.5149 - out0_loss: 0.1110 - out1_loss: 1.3928 - out2_loss: 1.1591 - out3_loss: 0.1979 - out4_loss: 0.0647 - out5_loss: 0.1397 - out6_loss: 0.1248 - out7_loss: 0.1199 - out8_loss: 0.1051 - out9_loss: 0.1054 - out10_loss: 0.1074 - out11_loss: 0.0848 - out0_mse: 0.2219 - out1_accuracy: 0.4114 - out2_accuracy: 0.5029 - out3_mse: 0.3968 - out4_mse: 0.1325 - out5_mse: 0.2993 - out6_mse: 0.2633 - out7_mse: 0.2398 - out8_mse: 0.2119 - out9_mse: 0.2113 - out10_mse: 0.2148 - out11_mse: 0.1713
Epoch 58/500
3/3 [==============================] - 0s 10ms/step - loss: 5.4334 - out0_loss: 0.1110 - out1_loss: 1.3940 - out2_loss: 1.1179 - out3_loss: 0.1898 - out4_loss: 0.0599 - out5_loss: 0.1246 - out6_loss: 0.1058 - out7_loss: 0.1259 - out8_loss: 0.0972 - out9_loss: 0.1100 - out10_loss: 0.1125 - out11_loss: 0.0885 - out0_mse: 0.2219 - out1_accuracy: 0.4343 - out2_accuracy: 0.5257 - out3_mse: 0.3796 - out4_mse: 0.1200 - out5_mse: 0.2531 - out

...

3/3 [==============================] - 0s 9ms/step - loss: 1.6972 - out0_loss: 0.1110 - out1_loss: 0.3407 - out2_loss: 0.5604 - out3_loss: 0.0272 - out4_loss: 0.0141 - out5_loss: 0.0219 - out6_loss: 0.0205 - out7_loss: 0.0132 - out8_loss: 0.0131 - out9_loss: 0.0097 - out10_loss: 0.0158 - out11_loss: 0.0186 - out0_mse: 0.2219 - out1_accuracy: 0.8857 - out2_accuracy: 0.7886 - out3_mse: 0.0544 - out4_mse: 0.0281 - out5_mse: 0.0439 - out6_mse: 0.0410 - out7_mse: 0.0264 - out8_mse: 0.0262 - out9_mse: 0.0193 - out10_mse: 0.0316 - out11_mse: 0.0372
Epoch 478/500
3/3 [==============================] - 0s 9ms/step - loss: 1.6822 - out0_loss: 0.1110 - out1_loss: 0.3058 - out2_loss: 0.5739 - out3_loss: 0.0268 - out4_loss: 0.0147 - out5_loss: 0.0245 - out6_loss: 0.0212 - out7_loss: 0.0161 - out8_loss: 0.0134 - out9_loss: 0.0092 - out10_loss: 0.0176 - out11_loss: 0.0184 - out0_mse: 0.2219 - out1_accuracy: 0.9143 - out2_accuracy: 0.7829 - out3_mse: 0.0536 - out4_mse: 0.0295 - out5_mse: 0.0490 - out6

3/3 [==============================] - 0s 9ms/step - loss: 1.7201 - out0_loss: 0.1110 - out1_loss: 0.3651 - out2_loss: 0.5759 - out3_loss: 0.0279 - out4_loss: 0.0125 - out5_loss: 0.0212 - out6_loss: 0.0207 - out7_loss: 0.0137 - out8_loss: 0.0135 - out9_loss: 0.0099 - out10_loss: 0.0170 - out11_loss: 0.0192 - out0_mse: 0.2219 - out1_accuracy: 0.8743 - out2_accuracy: 0.7771 - out3_mse: 0.0558 - out4_mse: 0.0250 - out5_mse: 0.0423 - out6_mse: 0.0414 - out7_mse: 0.0274 - out8_mse: 0.0269 - out9_mse: 0.0198 - out10_mse: 0.0340 - out11_mse: 0.0384
Epoch 492/500
3/3 [==============================] - 0s 8ms/step - loss: 1.5972 - out0_loss: 0.1110 - out1_loss: 0.3062 - out2_loss: 0.5253 - out3_loss: 0.0231 - out4_loss: 0.0122 - out5_loss: 0.0175 - out6_loss: 0.0227 - out7_loss: 0.0136 - out8_loss: 0.0126 - out9_loss: 0.0100 - out10_loss: 0.0159 - out11_loss: 0.0159 - out0_mse: 0.2219 - out1_accuracy: 0.9143 - out2_accuracy: 0.7886 - out3_mse: 0.0462 - out4_mse: 0.0245 - out5_mse: 0.0350 - out6

6/6 [==============================] - 0s 1ms/step - loss: 0.1344 - accuracy: 0.9771
Epoch 24/200
6/6 [==============================] - 0s 2ms/step - loss: 0.1317 - accuracy: 0.9771
Epoch 25/200
6/6 [==============================] - 0s 2ms/step - loss: 0.1290 - accuracy: 0.9771
Epoch 26/200
6/6 [==============================] - 0s 1ms/step - loss: 0.1266 - accuracy: 0.9771
Epoch 27/200
6/6 [==============================] - 0s 1ms/step - loss: 0.1243 - accuracy: 0.9771
Epoch 28/200
6/6 [==============================] - 0s 1ms/step - loss: 0.1232 - accuracy: 0.9771
Epoch 29/200
6/6 [==============================] - 0s 1ms/step - loss: 0.1204 - accuracy: 0.9829
Epoch 30/200
6/6 [==============================] - 0s 1ms/step - loss: 0.1190 - accuracy: 0.9829
Epoch 31/200
6/6 [==============================] - 0s 2ms/step - loss: 0.1176 - accuracy: 0.9771
Epoch 32/200
6/6 [==============================] - 0s 2ms/step - loss: 0.1160 - accuracy: 0.9829
Epoch 33/200
6/6 [===============

6/6 [==============================] - 0s 1ms/step - loss: 0.0844 - accuracy: 0.9829
Epoch 107/200
6/6 [==============================] - 0s 1ms/step - loss: 0.0845 - accuracy: 0.9829
Epoch 108/200
6/6 [==============================] - 0s 1ms/step - loss: 0.0843 - accuracy: 0.9829
Epoch 109/200
6/6 [==============================] - 0s 1ms/step - loss: 0.0842 - accuracy: 0.9829
Epoch 110/200
6/6 [==============================] - 0s 1ms/step - loss: 0.0838 - accuracy: 0.9829
Epoch 111/200
6/6 [==============================] - 0s 1ms/step - loss: 0.0839 - accuracy: 0.9829
Epoch 112/200
6/6 [==============================] - 0s 1ms/step - loss: 0.0841 - accuracy: 0.9829
Epoch 113/200
6/6 [==============================] - 0s 1ms/step - loss: 0.0836 - accuracy: 0.9829
Epoch 114/200
6/6 [==============================] - 0s 1ms/step - loss: 0.0834 - accuracy: 0.9829
Epoch 115/200
6/6 [==============================] - 0s 1ms/step - loss: 0.0834 - accuracy: 0.9829
Epoch 116/200
6/6 [=====

6/6 [==============================] - 0s 1ms/step - loss: 0.0765 - accuracy: 0.9829
Epoch 189/200
6/6 [==============================] - 0s 1ms/step - loss: 0.0763 - accuracy: 0.9829
Epoch 190/200
6/6 [==============================] - 0s 1ms/step - loss: 0.0763 - accuracy: 0.9829
Epoch 191/200
6/6 [==============================] - 0s 1ms/step - loss: 0.0762 - accuracy: 0.9829
Epoch 192/200
6/6 [==============================] - 0s 1ms/step - loss: 0.0762 - accuracy: 0.9829
Epoch 193/200
6/6 [==============================] - 0s 1ms/step - loss: 0.0763 - accuracy: 0.9829
Epoch 194/200
6/6 [==============================] - 0s 1ms/step - loss: 0.0760 - accuracy: 0.9829
Epoch 195/200
6/6 [==============================] - 0s 1ms/step - loss: 0.0759 - accuracy: 0.9829
Epoch 196/200
6/6 [==============================] - 0s 1ms/step - loss: 0.0759 - accuracy: 0.9829
Epoch 197/200
6/6 [==============================] - 0s 1ms/step - loss: 0.0759 - accuracy: 0.9829
Epoch 198/200
6/6 [=====

## Conclusion
You have now:
1. **Loaded and prepared** your data
2. **Trained** the OmicScoreModel on intermediate targets
3. **Transformed** those intermediate predictions to final labels
4. **Evaluated** and **(optionally) adjusted** model weights

Feel free to experiment with different architectures, hyperparameters (learning rate, batch size, etc.), or data splits. Modify **BioMapAI.py** to suit your specific needs.

**Happy modeling!**

In [7]:
end_time = time.time()
print(f"Total execution time: {end_time - start_time:.2f} seconds")

Total execution time: 32.90 seconds
